In [1]:
# Feature Importance를 살펴보는 것은 모델링 과정에서 어떤 특성이 모델 예측에 미치는 영향이 큰지 파악하는 데 도움을 주며, 
# 모델 해석과 선택에 있어서 유용한 정보를 제공할 수 있습니다. 
# 따라서 회귀 계수가 높은 값이면서 Feature Importance가 높다면 해당 특성이 모델에서 중요하다고 볼 수 있습니다.

In [2]:
# 재현율(Recall)은 이진 분류 모델의 평가 지표 중 하나로, 실제 양성 케이스(Positive cases) 중에서 모델이 정확하게 양성으로 예측한 케이스의 비율
# 정밀도(Precision)는 이진 분류 모델의 평가 지표 중 하나로, 모델이 예측한 양성 케이스 중에서 실제 양성인 케이스의 비율

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.svm as svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

In [4]:
train_df = pd.read_csv('../EDA/scaled_data.csv')
train_df.head(1)

,Unnamed: 0,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입
0,0,0,2,1,1,3,0,0,3,1,0,1,4,5,1,2,3,0


In [5]:
train_copy = train_df.copy()

In [6]:
# 직업군과 산업컬럼을 겹쳐서 카테고리컬로
# 카테고리로 할 거면 원핫이코딩으로 헤줘야하고(스케일링 하면 안 됨)
# LogisticRegression 돌렸을 때 회귀계수 크게 나온다면 연체여부에 영향을 많이 끼치는 것
# 카테고리로 안 주고 실수값으로 밸류값을 지정해도 됨

In [7]:
job_ind = pd.read_csv('../EDA/job_ind.csv')
job_ind.drop(job_ind.index[-1], inplace=True)
job_ind

,직업,산업군,total,default
0,1,5.0,0.014818,0.138323
1,1,16.0,0.005917,0.055236
2,1,24.0,0.001533,0.014315
3,1,17.0,0.000933,0.008713
4,1,26.0,0.000700,0.006535
...,...,...,...,...
459,0,26.0,0.000000,0.000000
460,0,31.0,0.000000,0.000000
461,0,11.0,0.000000,0.000000
462,0,0.0,0.000000,0.000000


In [8]:
job_ind['산업군'] = job_ind['산업군'].astype(int)
job_ind['직업'] = job_ind['직업'].astype(int)

In [9]:
job_ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   직업       464 non-null    int32  
 1   산업군      464 non-null    int32  
 2   total    464 non-null    float64
 3   default  464 non-null    float64
dtypes: float64(2), int32(2)
memory usage: 11.0 KB


In [10]:
job_ind.head()

,직업,산업군,total,default
0,1,5,0.014818,0.138323
1,1,16,0.005917,0.055236
2,1,24,0.001533,0.014315
3,1,17,0.000933,0.008713
4,1,26,0.000700,0.006535


In [11]:
# 산업군_직업 컬럼 추가 (total를 value값으로 대입)
for i in train_copy['산업군'].unique():
    for j in train_copy['직업'].unique():
        # # job_ind 데이터프레임에서 조건에 맞는 total 값을 가져옴
        value = job_ind.loc[(job_ind['산업군'] == i) & (job_ind['직업'] == j), 'total'].values

        # train_copy에서 해당 조건에 맞는 행에 새로운 컬럼 '산업군_직업'을 만들어 value 값을 할당
        train_copy.loc[(train_copy['산업군'] == i) & (train_copy['직업'] == j), '산업군_직업'] = value[0] if len(value) > 0 else None

train_copy


,Unnamed: 0,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입,산업군_직업
0,0,0,2,1,1,3,0,0,3,1,0,1,4,5,1,2,3,0,0.014818
1,1,0,1,1,0,1,1,0,3,1,0,4,2,16,2,1,1,3,0.001100
2,2,0,2,0,1,1,0,0,3,1,0,5,2,16,1,1,1,1,0.010751
3,3,0,2,1,0,2,0,0,3,1,0,1,3,17,1,1,3,2,0.000933
4,4,0,2,0,1,1,0,0,3,1,0,1,2,16,1,0,0,1,0.005917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,59995,0,2,0,1,3,0,1,3,1,0,1,1,5,3,2,1,0,0.014818
59985,59996,1,1,1,1,1,0,0,3,1,0,12,2,24,2,0,1,3,0.002567
59986,59997,1,2,0,0,1,1,1,3,1,0,1,1,22,0,1,1,2,0.000100
59987,59998,0,2,1,1,1,0,3,3,1,0,17,1,19,2,1,1,1,0.000233


In [12]:
train_copy[['산업군','직업','산업군_직업']]

,산업군,직업,산업군_직업
0,5,1,0.014818
1,16,4,0.001100
2,16,5,0.010751
3,17,1,0.000933
4,16,1,0.005917
...,...,...,...
59984,5,1,0.014818
59985,24,12,0.002567
59986,22,1,0.000100
59987,19,17,0.000233


In [13]:
# 산업군_직업2 컬럼 추가 (default를 value값으로 대입)
for i in train_copy['산업군'].unique():
    for j in train_copy['직업'].unique():
        # # job_ind 데이터프레임에서 조건에 맞는 total 값을 가져옴
        value = job_ind.loc[(job_ind['산업군'] == i) & (job_ind['직업'] == j), 'default'].values

        # train_copy에서 해당 조건에 맞는 행에 새로운 컬럼 '산업군_직업'을 만들어 value 값을 할당
        train_copy.loc[(train_copy['산업군'] == i) & (train_copy['직업'] == j), '산업군_직업2'] = value[0] if len(value) > 0 else None

train_copy

,Unnamed: 0,TARGET,성별,차량 소유 여부,부동산 소유 여부,수입 유형,최종 학력,결혼 여부,주거 형태,휴대전화 소유 여부,이메일 소유 여부,직업,가족 구성원 수,산업군,나이,가입연수,도시구분,월간 수입,산업군_직업,산업군_직업2
0,0,0,2,1,1,3,0,0,3,1,0,1,4,5,1,2,3,0,0.014818,0.138323
1,1,0,1,1,0,1,1,0,3,1,0,4,2,16,2,1,1,3,0.001100,0.010269
2,2,0,2,0,1,1,0,0,3,1,0,5,2,16,1,1,1,1,0.010751,0.100358
3,3,0,2,1,0,2,0,0,3,1,0,1,3,17,1,1,3,2,0.000933,0.008713
4,4,0,2,0,1,1,0,0,3,1,0,1,2,16,1,0,0,1,0.005917,0.055236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59984,59995,0,2,0,1,3,0,1,3,1,0,1,1,5,3,2,1,0,0.014818,0.138323
59985,59996,1,1,1,1,1,0,0,3,1,0,12,2,24,2,0,1,3,0.002567,0.023961
59986,59997,1,2,0,0,1,1,1,3,1,0,1,1,22,0,1,1,2,0.000100,0.000934
59987,59998,0,2,1,1,1,0,3,3,1,0,17,1,19,2,1,1,1,0.000233,0.002178


In [14]:
train_copy[['산업군','직업','산업군_직업2']]

,산업군,직업,산업군_직업2
0,5,1,0.138323
1,16,4,0.010269
2,16,5,0.100358
3,17,1,0.008713
4,16,1,0.055236
...,...,...,...
59984,5,1,0.138323
59985,24,12,0.023961
59986,22,1,0.000934
59987,19,17,0.002178


In [15]:
# feature와 label 분리
feature = train_copy.drop('TARGET', axis=1)
label = train_copy['TARGET']

In [16]:
# train test 분리
X_train_old, X_test, y_train_old , y_test = train_test_split(feature, label, test_size=0.3 , random_state=42)

In [17]:
y_train_old.value_counts()

TARGET
0    37465
1     4527
Name: count, dtype: int64

In [18]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

# 언더샘플링 수행
rus = RandomUnderSampler(random_state=42)
X_train, y_train = rus.fit_resample(X_train_old, y_train_old)

# 언더샘플링 후의 클래스 분포 확인
print("클래스 분포 (언더샘플링 전):", len(y_train_old), sum(y_train_old == 0), sum(y_train_old == 1))
print("클래스 분포 (언더샘플링 후):", len(y_train), sum(y_train == 0), sum(y_train == 1))

클래스 분포 (언더샘플링 전): 41992 37465 4527
클래스 분포 (언더샘플링 후): 9054 4527 4527


In [19]:
y_train.value_counts()

TARGET
0    4527
1    4527
Name: count, dtype: int64

In [20]:
# 1. Decision Tree를 이용한 학습
# 1-1. GridSearchCV를 적용해 Decision Tree의 교차검증 및 하이퍼파라미터 튜닝

dt = DecisionTreeClassifier()

parameters = {'max_depth' : [2, 3, 4, 5],
             'min_samples_split' : [1, 3, 5, 7, 9]}

grid_dt = GridSearchCV(dt, param_grid = parameters, cv=3, refit=True)
grid_dt.fit(X_train, y_train)

dt = grid_dt.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_dt.best_params_}")
print(f"최고 예측 정확도: {grid_dt.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 4, 'min_samples_split': 3}
최고 예측 정확도: 0.5896


In [21]:
# Decision Tree 적용
pred = dt.predict(X_test)
pred_proba = dt.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
f1score = f1_score(y_test, pred)
auc_score = roc_auc_score(y_test , pred_proba_1)
print(f'의사결정나무 정확도 : {acc:.3f}')
print(f'의사결정나무 정밀도 : {prec:.3f}')
print(f'의사결정나무 재현율 : {rec:.3f}')
print(f'의사결정나무 f1_score : {f1score:.3f}')
print(f'의사결정나무 roc_auc : {auc_score:.3f}')

의사결정나무 정확도 : 0.651
의사결정나무 정밀도 : 0.158
의사결정나무 재현율 : 0.532
의사결정나무 f1_score : 0.244
의사결정나무 roc_auc : 0.619


In [22]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, dt.feature_importances_):
    print(f"{feature}: {importance}")

Unnamed: 0: 0.0
성별: 0.16624445355155812
차량 소유 여부: 0.037082879140233116
부동산 소유 여부: 0.0
수입 유형: 0.052529389545761
최종 학력: 0.2996032645992076
결혼 여부: 0.08488266716978589
주거 형태: 0.0
휴대전화 소유 여부: 0.0
이메일 소유 여부: 0.0
직업: 0.0
가족 구성원 수: 0.0
산업군: 0.018807281071107358
나이: 0.28752785256246166
가입연수: 0.0
도시구분: 0.0
월간 수입: 0.0
산업군_직업: 0.053322212359885185
산업군_직업2: 0.0


In [23]:
# 2. RandomForest를 이용한 학습
# 2-1. GridSearchCV를 적용해 RandomForest의 교차검증 및 하이퍼파라미터 튜닝

rf = RandomForestClassifier()

parameters = {
    'max_depth' : [6, 8, 12],
    'min_samples_split' : [16, 24]
}

grid_rf = GridSearchCV(rf, param_grid = parameters, cv=3, refit=True)
grid_rf.fit(X_train, y_train)

rf = grid_rf.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_rf.best_params_}")
print(f"최고 예측 정확도: {grid_rf.best_score_:.4f}")

최적 하이퍼 파라미터: {'max_depth': 6, 'min_samples_split': 24}
최고 예측 정확도: 0.5991


In [24]:
# RandomForest 적용

rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
pred_proba = rf.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
f1score = f1_score(y_test, pred)
auc_score = roc_auc_score(y_test , pred_proba_1)

print(f'랜덤포레스트 정확도 : {acc:.3f}')
print(f'랜덤포레스트 정밀도 : {prec:.3f}')
print(f'랜덤포레스트 재현율 : {rec:.3f}')
print(f'랜덤포레스트 f1_score : {f1score:.3f}')
print(f'랜덤포레스트 roc_auc : {auc_score:.3f}')

랜덤포레스트 정확도 : 0.574
랜덤포레스트 정밀도 : 0.136
랜덤포레스트 재현율 : 0.566
랜덤포레스트 f1_score : 0.219
랜덤포레스트 roc_auc : 0.597


In [25]:
# 트리 기반 모델의 특성 중요도 확인
for feature, importance in zip(X_train.columns, rf.feature_importances_):
    print(f"{feature}: {importance}")

Unnamed: 0: 0.2343745003770958
성별: 0.02221643456473473
차량 소유 여부: 0.02658801178159196
부동산 소유 여부: 0.03011380386621668
수입 유형: 0.032747682396615165
최종 학력: 0.030419575101654554
결혼 여부: 0.04164414906402302
주거 형태: 0.01820326727778064
휴대전화 소유 여부: 0.0
이메일 소유 여부: 0.011887172092997453
직업: 0.055634521818899614
가족 구성원 수: 0.05319374392119545
산업군: 0.05983254890536905
나이: 0.05944962591096688
가입연수: 0.05196959497001787
도시구분: 0.055570406949712645
월간 수입: 0.06853956686164736
산업군_직업: 0.0730410796425252
산업군_직업2: 0.07457431449695606


In [26]:
# 3. Logistic regrssion 을 이용한 학습
# 3-1. GridSearchCV를 적용해 Logistic regrssion의 교차검증 및 하이퍼파라미터 튜닝

lr = LogisticRegression()

parameters = {'penalty': ['l2','l1'],
          'C':[0.01,0.1,1,10]}

grid_lr = GridSearchCV(lr, param_grid = parameters, cv=3, refit=True)
grid_lr.fit(X_train, y_train)

lr = grid_lr.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_lr.best_params_}")
print(f"최고 예측 정확도: {grid_lr.best_score_:.4f}")

최적 하이퍼 파라미터: {'C': 0.01, 'penalty': 'l2'}
최고 예측 정확도: 0.5235


In [27]:
#3-2. Logistic Regression 적용
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
pred_proba = lr.predict_proba(X_test)
pred_proba_1 = pred_proba[:, 1]

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred_proba_1)
f1score = f1_score(y_test, pred)

print(f'Logistic regrssion 정확도 : {acc:.3f}')
print(f'Logistic regrssion 정밀도 : {prec:.3f}')
print(f'Logistic regrssion 재현율 : {rec:.3f}')
print(f'Logistic regrssion f1_score : {f1score:.3f}')
print(f'Logistic regrssion roc_auc : {auc_score:.3f}')

Logistic regrssion 정확도 : 0.894
Logistic regrssion 정밀도 : 0.000
Logistic regrssion 재현율 : 0.000
Logistic regrssion f1_score : 0.000
Logistic regrssion roc_auc : 0.504


In [28]:
# 학습된 모델의 특성 중요도 확인
feature_importance = lr.coef_[0]  # 특성의 가중치 또는 중요도

# 특성별 중요도 출력
for feature, importance in zip(X_train.columns, feature_importance):
    print(f"{feature}: {importance}")

Unnamed: 0: -5.190031575669914e-07
성별: -1.4967003324349563e-10
차량 소유 여부: -4.721519490122864e-11
부동산 소유 여부: 1.4561127926775443e-11
수입 유형: -2.0778712161680485e-10
최종 학력: -1.2848803534708883e-10
결혼 여부: 1.6601635255419224e-10
주거 형태: -5.538956930270597e-12
휴대전화 소유 여부: 1.2894507519238146e-11
이메일 소유 여부: 2.1266042541693702e-12
직업: 6.327956964878778e-10
가족 구성원 수: 1.0265490112810232e-10
산업군: 1.2692198451272004e-09
나이: -5.012112360302582e-10
가입연수: -1.5236339602655733e-10
도시구분: 1.0153694623075922e-10
월간 수입: -8.662965477558813e-11
산업군_직업: -9.372818647285585e-13
산업군_직업2: -8.74951956658759e-12


In [29]:
# 5. KNN 을 이용한 학습
# 5-1. GridSearchCV를 적용해 KNN의 교차검증 및 하이퍼파라미터 튜닝

knn = KNeighborsClassifier()

parameters = {'n_neighbors': [3, 5, 7, 9],
              'weights': ['uniform', 'distance']
          }

grid_knn = GridSearchCV(knn, param_grid = parameters, cv=3, refit=True)
grid_knn.fit(X_train, y_train)

knn = grid_knn.best_estimator_

print(f"최적 하이퍼 파라미터: {grid_knn.best_params_}")
print(f"최고 예측 정확도: {grid_knn.best_score_:.4f}")

최적 하이퍼 파라미터: {'n_neighbors': 9, 'weights': 'distance'}
최고 예측 정확도: 0.5043


In [30]:
# 5-2 knn 적용
knn = KNeighborsClassifier()

knn.fit(X_train, y_train)
pred = knn.predict(X_test) 

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)

print(f'KNN 정확도 : {acc:.3f}')
print(f'KNN 정밀도 : {prec:.3f}')
print(f'KNN 재현율 : {rec:.3f}')
print(f'KNN f1_score : {f1score:.3f}')
print(f'KNN roc_auc : {auc_score:.3f}')

KNN 정확도 : 0.493
KNN 정밀도 : 0.105
KNN 재현율 : 0.503
KNN f1_score : 0.173
KNN roc_auc : 0.498


In [31]:
# 6. xgboost 를 이용한 학습
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators = 150,
                            learning_rate = 0.2,
                            max_depth = 10,
                            min_child_weight = 5,
                            gamma = 10)

# fit
xgb_model.fit(X_train, y_train)

# 예측
y_pred = xgb_model.predict(X_test)


# 모델 성능평가
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_test, y_pred)

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)


# score print
print(f'xgboost 정확도 : {acc:.3f}')
print(f'xgboost 정밀도 : {prec:.3f}')
print(f'xgboost 재현율 : {rec:.3f}')
print(f'xgboost f1_score : {f1score:.3f}')
print(f'xgboost roc_auc : {auc_score:.3f}')

xgboost 정확도 : 0.493
xgboost 정밀도 : 0.105
xgboost 재현율 : 0.503
xgboost f1_score : 0.173
xgboost roc_auc : 0.498


In [ ]:
import sklearn.svm as svm

svm = svm.SVC(kernel = 'rbf', random_state=100)
parameters = {'C': [0.001, 0.01, 0.1, 1],
                 'gamma':[0.001, 0.01, 0.1, 1, 10, 25]}

grid_svm = GridSearchCV(svm, param_grid = parameters, cv=5, refit=True)
grid_svm.fit(X_train, y_train)

result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

In [ ]:
# 5-2 svm 적용
svm = KNeighborsClassifier()

svm.fit(X_train, y_train)
pred = svm.predict(X_test) 

acc = accuracy_score(y_test , pred)
prec = precision_score(y_test , pred)
rec = recall_score(y_test , pred)
auc_score = roc_auc_score(y_test , pred)
f1score = f1_score(y_test, pred)

print(f'svm 정확도 : {acc:.3f}')
print(f'svm 정밀도 : {prec:.3f}')
print(f'svm 재현율 : {rec:.3f}')
print(f'svm f1_score : {f1score:.3f}')
print(f'svm roc_auc : {auc_score:.3f}')